In [6]:
import astropy.units as u
import math
from astropy import coordinates

In [7]:
# Position of M51 - https://ned.ipac.caltech.edu/byname?objname=M51&hconst=67.8&omegam=0.308&omegav=0.692&wmap=4&corr_z=1
ra = 202.484167 *u.deg
dec = 47.230556 *u.deg
coords = coordinates.SkyCoord(ra, dec, frame='icrs')
coords

<SkyCoord (ICRS): (ra, dec) in deg
    (202.484167, 47.230556)>

In [8]:
coords.to_string('hmsdms') # Print string in hmsdms format

'13h29m56.20008s +47d13m50.0016s'

In [9]:
coords.ra.value, coords.dec.value # Print as floats

(202.484167, 47.230556)

In [10]:
coords.fk5 # Print in FK5 format

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (202.48416679, 47.23055578)>

In [11]:
coords.galactic # Print in Galactic format

<SkyCoord (Galactic): (l, b) in deg
    (104.86604847, 68.52442261)>

In [12]:
box = '01 24 03' # Define box
box_deg = box.split(' ')
box_deg = [float(i) for i in box_deg]
box_deg = box_deg[0] *u.deg + box_deg[1] *u.arcmin + box_deg[2] *u.arcsecond
print('Box original:', box)
print('Box in degrees:', box_deg)

Box original: 01 24 03
Box in degrees: 1.4008333333333332 deg


In [73]:
# def wdb_sql_box explained:
# coords sets the ra0 and dec0 of the point T (the target) around which a "box" is drawn
# the box in wdb is defined as a range in ra and in dec, 
# that is, it is the intersection between:
#    a spherical lune between two great circles passing through the equatorial poles, 
#    of amplitude equal to box_deg at the position of T, wand hose central meridian is at ra=ra0 
# and 
#    the spherical surface between two lines of equal declinations (parallels) separated by the box_deg angular distance, 
#    and centered onto dec=dec0
#
# Once projected onto the plane x=RA and y=DEC such box is expressed by the simple condition:
# ra between ra_min and ra_max and dec between dec_min and dec_max.
#
# For the declination it is easy to compute: dec_min = dec0 - box_deg/2  and dec_max = dec0 + box_deg/2
#
# For the right ascension, an approximation is used, valid for small box_deg values: 
# the triangle with sides, (a) the ra0 meridian from T to the closest pole (P), 
#                          (b) the parallel from T=(ra0,dec0) to W=(ra0+ra_max,dec0)
#                          (c) the meridian from W to P
# is approximated to be a spherical triangle (it is not, as a parallel is not a great circle).
# With this approximation, using the sin theorem it is true that: sin(90-dec0) / sin(90) = sin( box_deg/2 ) / sin( delta_alpha/2 )
# where delta_alpha/2 is half of the amplitude of the lune at the equator.
#
# This gives: delta_alpha/2 = asin( sin(box_size/2) / cos(dec0) )
# The box is therefore defined with the constraint: RA between ra0 - delta_alpha/2 and ra0 + delta_alpha/2 AND DEC between dec0 - box_deg/2 and dec0 + box_deg/2
# or said in different words: 
#      dec_min = dec0 - box_deg/2      dec_max = dec0 + box_deg/2
#      ra_min  = ra0  + delta_alpha/2  ra_max  = ra0  + delta_alpha/2
#      with half_delta_alpha = asin( sin(box_size/2) / cos(dec0) )
#
# BUT: if the box extends over the closest pole, or over the meridian ra=0, the above constraint miserably fails.
# It is important to protect against this, and this is taken care within the function.
#
def wdb_sql_box(coords, box_deg):
    
    dec_min = coords.dec.value - box_deg.value/2
    dec_max = coords.dec.value + box_deg.value/2

    no_ra_range = 0 # if = 1 the condition in ra is to be dropped, as the box goes over the pole
    if dec_min <= -90:
        dec_min = -90
        no_ra_range = 1
    if dec_max >= 90:
        dec_max = 90
        no_ra_range = 1

    if no_ra_range:
        ra_min = 0
        ra_max = 360
    else:
        sin_half_delta_alpha = math.sin(box_deg.value/2*math.pi/180.)/math.cos(coords.dec.value*math.pi/180.)
        # notice that here dec cannot be +/- 90, hence the division by cos(dec) will never explode
        # though better to protect against floating point errors:
        if sin_half_delta_alpha > 1.0:
            sin_half_delta_alpha = 1.0
        if sin_half_delta_alpha < -1.0:
            sin_half_delta_alpha = -1.0
            
        half_delta_alpha = math.asin( sin_half_delta_alpha ) * 180./math.pi
        ra_min  = coords.ra.value  - half_delta_alpha
        ra_max  = coords.ra.value  + half_delta_alpha
        if ra_min < 0:
            ra_min += 360
        if ra_max > 360:
            ra_max -= 360

    if ra_min > ra_max:
        ra_constraint  = f'(RA >= {ra_min} OR RA <= {ra_max})'
    else: 
        ra_constraint = f'RA between {ra_min} and {ra_max}'

    dec_constraint = f'DEC between {dec_min} and {dec_max}'

    sql_constraint = ra_constraint + ' AND ' + dec_constraint
    return(sql_constraint)
    # return(f'RA between {ra_min} and {ra_max} AND DEC between {dec_min} and {dec_max}')

In [49]:
print(get_sql_box(coords, box_deg))

RA between 201.4662129040896 and 203.52938438629218 AND DEC between 46.52557178225933 and 47.92628278338335


In [74]:
box = '01 24 03' # Define box
box_deg = box.split(' ')
box_deg = [float(i) for i in box_deg]
box_deg = box_deg[0] *u.deg + box_deg[1] *u.arcmin + box_deg[2] *u.arcsecond
print('Box original:', box)
print('Box in degrees:', box_deg)

print(wdb_sql_box(coords, box_deg))

ra = 202.484167 *u.deg
dec = 89.530556 *u.deg
coordspole = coordinates.SkyCoord(ra, dec, frame='icrs')
print(wdb_sql_box(coordspole, box_deg))

ra = 0.484167 *u.deg
dec = 19.230556 *u.deg
coordsra0 = coordinates.SkyCoord(ra, dec, frame='icrs')
print(wdb_sql_box(coordsra0, box_deg))

ra = 359.784167 *u.deg
dec = 0 *u.deg
coordsra0 = coordinates.SkyCoord(ra, dec, frame='icrs')
print(wdb_sql_box(coordsra0, box_deg))

box_deg = 1.0001*u.deg
coords89 = coordinates.SkyCoord(10*u.deg, 89.499999999*u.deg, frame='icrs')
print(wdb_sql_box(coords89, box_deg))


Box original: 01 24 03
Box in degrees: 1.4008333333333332 deg
RA between 201.45267130343717 and 203.51566269656286 AND DEC between 46.53013933333333 and 47.93097266666667
RA between 0 and 360 AND DEC between 88.83013933333334 and 90
(RA >= 359.74235577487804 OR RA <= 1.2259782251219886) AND DEC between 18.530139333333334 and 19.930972666666666
(RA >= 359.08375033333334 OR RA <= 0.48458366666670827) AND DEC between -0.7004166666666666 and 0.7004166666666666
RA between 0 and 360 AND DEC between 88.999949999 and 90


In [75]:
# Crossing the pole
print(wdb_sql_box(coordinates.SkyCoord(0*u.deg,-89.5*u.deg, frame='icrs'), box_deg=2*u.deg))

RA between 0 and 360 AND DEC between -90 and -88.5
